In [ ]:
!pip install pyradox-tabular -q
import pandas as pd
import numpy as np
import sklearn
from pyradox_tabular.data import DataLoader

from pyradox_tabular.model_config import NeuralObliviousDecisionEnsembleConfig
from pyradox_tabular.nn import NeuralObliviousDecisionEnsemble
from pyradox_tabular.data import DataLoader
from pyradox_tabular.data_config import DataConfig
from pyradox_tabular.nn import TabTransformer
from tensorflow import keras
from tensorflow.keras import layers


from pyradox_tabular.model_config import DeepAndCrossNetworkConfig
from pyradox_tabular.nn import DeepAndCrossTabularNetwork

In [ ]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")

In [ ]:
features = [c for c in train.columns if c not in ("target","row_id")]
target="target"
x_test=test[features]
y = train[target]
X = train[features]
numerical_features = X.columns

In [ ]:
from sklearn.utils import shuffle
train = shuffle(train)

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X,
                                                      y,
                                                      stratify=y,
                                                      test_size=0.20,
                                                      random_state=2020)

In [ ]:
y_train.value_counts()

In [ ]:
y_valid.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
y_train = le.fit_transform(y_train)

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

In [ ]:
y_valid = le.fit_transform(y_valid)

In [ ]:
unique, counts = np.unique(y_valid, return_counts=True)
dict(zip(unique, counts))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
oe = OneHotEncoder()

In [ ]:
y_train = oe.fit_transform(y_train.reshape(-1, 1))

In [ ]:
y_train

In [ ]:
y_train = y_train.toarray()

In [ ]:
y_train

In [ ]:
y_train =pd.DataFrame(y_train)

In [ ]:
y_train

In [ ]:
y_valid = oe.fit_transform(y_valid.reshape(-1, 1))
y_valid = y_valid.toarray()
y_valid =pd.DataFrame(y_valid)

In [ ]:
y_valid

In [ ]:
X_train

In [ ]:
X_valid

In [ ]:
X_train.shape[0]+X_valid.shape[0] == X.shape[0]

In [ ]:
y_train.shape[0]+y_valid.shape[0] == y.shape[0]

In [ ]:
from sklearn.preprocessing import RobustScaler
RobustScaler_transformer = RobustScaler().fit(X_train.values)

In [ ]:
X_trainRobustScaler = RobustScaler_transformer.transform(X_train.values)

In [ ]:
X_validRobustScaler = RobustScaler_transformer.transform(X_valid.values)

In [ ]:
X_testRobustScaler = RobustScaler_transformer.transform(x_test.values)

In [ ]:
X_train =pd.DataFrame(
    X_trainRobustScaler,
    columns=list(numerical_features))

In [ ]:
X_valid =pd.DataFrame(
    X_validRobustScaler,
    columns=list(numerical_features))

In [ ]:
x_test =pd.DataFrame(
    X_testRobustScaler,
    columns=list(numerical_features))

In [ ]:
data_config = DataConfig(numeric_feature_names=list(numerical_features), 
                         categorical_features_with_vocabulary={}
                        )

In [ ]:
data_train = DataLoader.from_df(X_train, y_train, batch_size=1024)
data_valid = DataLoader.from_df(X_valid, y_valid, batch_size=1024)
data_test = DataLoader.from_df(x_test, batch_size=1024)

In [ ]:
# class DeepAndCrossNetworkConfig:
#     """Configurations for the deep and cross network.
#     Args:
#         num_outputs (int): number of cells in output layer.
#         out_activation (str): activation function for output layer.
#         hidden_units (list): list of hidden units for each hidden layer.
#         n_cross (int, optional): number of cross features, Defaults to 2.
#         dropout_rate (float, optional): dropout rate, Defaults to 0.3.
#         use_embeddings (bool, optional): whether to use embeddings, Defaults to True.
#         embedding_dim (int, optional): embedding dimension, Defaults to 32.
#     """

#     def __init__(
#         self,
#         num_outputs,
#         out_activation,
#         hidden_units,
#         n_cross=2,
#         dropout_rate=0.3,
#         use_embeddings=True,
#         embedding_dim=32,
#     ):
#         self.NUM_OUT = num_outputs
#         self.OUT_ACTIVATION = out_activation
#         self.HIDDEN_UNITS = hidden_units
#         self.DROPOUT_RATE = dropout_rate
#         self.USE_EMBEDDINGS = use_embeddings
#         self.EMBEDDING_DIM = embedding_dim
#         self.NUM_CROSS_LAYERS = 

# https://github.com/Ritvik19/pyradox-tabular/blob/a642ba2c6196a93611c3b5d32042b63ce862ac94/pyradox_tabular/model_config.py#L46

In [ ]:
model_config = DeepAndCrossNetworkConfig(num_outputs=10,
                                         out_activation='softmax',
                                         hidden_units=[64, 64],
                                         n_cross=2,
                                         dropout_rate=0.8)

In [ ]:
model = DeepAndCrossTabularNetwork.from_config(data_config,
                                               model_config,
                                               name="deep_cross_network")

In [ ]:
import tensorflow as tf

lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                               patience=5, verbose=2)

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                           patience=20, 
                           verbose=1,
                           mode="min", 
                           restore_best_weights=True)
        
    
callbacks = [lr,es]

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics ="categorical_accuracy" )

In [ ]:
model.fit(data_train,
          validation_data=data_valid,
          epochs=30,
          batch_size=1024,
          callbacks=callbacks)

In [ ]:
test_preds = model.predict(data_test,
                          batch_size=1024)

In [ ]:
test_preds_decoded = oe.inverse_transform(test_preds)

In [ ]:
test_preds_decoded

In [ ]:
test_preds_decoded_inversed = le.inverse_transform(test_preds_decoded)

In [ ]:
test_preds_decoded_inversed

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [ ]:
sub['target'] = test_preds_decoded_inversed

In [ ]:
sub.groupby(['target']).count()

In [ ]:
sub.to_csv('submission.csv', index=False)